In [2]:
!pip install pytubefix
!pip install moviepy
!pip install googletrans==3.1.0a0
!pip install openai-whisper
!pip install srt
!pip install tqdm
!pip install logging
!pip install git+https://github.com/m-bain/whisperx.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16353 sha256=7476799948496f2ffd7f3f4abb02ea5ec28f64ff0e4b352c1cfd9a41bf0f646a
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
 

In [1]:
import os
from pathlib import Path
from dataclasses import dataclass
from typing import Optional, List, Dict
from pytubefix import YouTube
from googletrans import Translator
from datetime import timedelta
import whisperx
import torch
import gc
import srt
import re
import logging
from tqdm import tqdm
import subprocess
import shutil

import warnings
warnings.filterwarnings('ignore')

@dataclass
class VideoProcessingConfig:
    """Configuration settings for video processing."""
    output_dir: Path
    temp_dir: Path
    video_quality: str = "720p"
    subtitle_font: str = "Arial"
    subtitle_size: int = 24
    subtitle_color: str = "white"
    subtitle_stroke_color: str = "black"
    subtitle_stroke_width: int = 1
    max_translation_retries: int = 3
    target_language: str = "es"
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    batch_size: int = 16 # Ajustar según la memoria GPU disponible

class ProcessingProgress:
    """Clase para manejar la visualización del progreso"""
    def __init__(self):
        self.current_step = ""
        self.pbar = None

    def update_step(self, step: str, total: Optional[int] = None):
        self.current_step = step
        if self.pbar is not None:
            self.pbar.close()
        if total is not None:
            self.pbar = tqdm(total=total, desc=step)
        print(f"\n>>> {step}")

    def update_progress(self, n: int = 1):
        if self.pbar is not None:
            self.pbar.update(n)

    def close(self):
        if self.pbar is not None:
            self.pbar.close()

class VideoProcessor:
    def __init__(self, config: VideoProcessingConfig):
        self.config = config
        self.setup_directories()
        self.setup_logging()
        self.translator = Translator()
        self.progress = ProcessingProgress()

    def setup_directories(self):
        """Create necessary directories if they don't exist."""
        self.config.output_dir.mkdir(parents=True, exist_ok=True)
        self.config.temp_dir.mkdir(parents=True, exist_ok=True)

    def setup_logging(self):
        """Configure logging with both file and console handlers."""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(self.config.output_dir / 'video_processor.log'),
                logging.StreamHandler()
            ]
        )

    @staticmethod
    def sanitize_filename(filename: str) -> str:
        """Clean filename of invalid characters."""
        return re.sub(r'[\\/*?:"<>|]', "", filename)

    def download_video(self, url: str, output_path: Path, filename: Optional[str] = None) -> Optional[Path]:
        """Download video with progress tracking and error handling."""
        try:
            self.progress.update_step("Descargando video...")

            # Configurar YouTube con opciones para evitar restricciones de edad
            yt = YouTube(url)

            # Intentar obtener el stream con la calidad deseada
            video = (yt.streams
                    .filter(progressive=True, file_extension='mp4')
                    .get_by_resolution(self.config.video_quality))

            # Si no se encuentra la calidad deseada, obtener la mejor calidad disponible
            if not video:
                video = (yt.streams
                        .filter(progressive=True, file_extension='mp4')
                        .order_by('resolution')
                        .desc()
                        .first())

            if not video:
                raise Exception("No se encontró ningún stream de video válido")

            # Establecer nombre de archivo
            if filename:
                filename = self.sanitize_filename(filename)
            else:
                filename = self.sanitize_filename(f"{yt.title}.mp4")

            output_file = output_path / filename

            # Descargar el video con barra de progreso
            print(f"Descargando: {yt.title}")
            video.download(output_path=str(output_path), filename=filename)
            print(f"Descarga completada: {output_file}")

            return output_file
        except Exception as e:
            logging.error(f"Error downloading video {url}: {str(e)}")
            return None

    def extract_audio(self, video_path: Path, output_path: Path) -> Optional[Path]:
        """Extract audio using ffmpeg."""
        try:
            self.progress.update_step("Extrayendo audio...")

            command = [
                'ffmpeg', '-i', str(video_path),
                '-vn', '-acodec', 'pcm_s16le',
                '-ar', '16000', '-ac', '1',
                '-y',  # Sobrescribir archivo si existe
                str(output_path)
            ]

            process = subprocess.Popen(
                command,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                universal_newlines=True
            )

            _, stderr = process.communicate()

            if process.returncode != 0:
                raise Exception(f"FFmpeg error: {stderr}")

            return output_path
        except Exception as e:
            logging.error(f"Error extracting audio from {video_path}: {str(e)}")
            return None

    def transcribe_audio(self, audio_path: Path) -> Optional[Dict]:
        """Transcribe audio using WhisperX."""
        try:
            self.progress.update_step("Transcribiendo audio...")

            # Cargar modelo de WhisperX
            model = whisperx.load_model(
                "large-v2",
                self.config.device,
                compute_type="float16" if self.config.device == "cuda" else "float32"
            )

            # Transcribir audio
            audio = whisperx.load_audio(str(audio_path))
            result = model.transcribe(
                audio,
                batch_size=self.config.batch_size
            )

            # Alinear timestamps con VAD
            model_a, metadata = whisperx.load_align_model(
                language_code=result["language"],
                device=self.config.device
            )

            result = whisperx.align(
                result["segments"],
                model_a,
                metadata,
                audio,
                self.config.device,
                return_char_alignments=False
            )

            # Liberar memoria
            del model
            del model_a
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            return result

        except Exception as e:
            logging.error(f"Error transcribing {audio_path}: {str(e)}")
            return None

    def translate_text(self, text: str) -> str:
        """Translate text with retry logic and error handling."""
        for attempt in range(self.config.max_translation_retries):
            try:
                translation = self.translator.translate(
                    text,
                    dest=self.config.target_language
                )
                return translation.text
            except Exception as e:
                if attempt == self.config.max_translation_retries - 1:
                    logging.error(f"Translation failed after {self.config.max_translation_retries} attempts: {str(e)}")
                    return text
                continue

    def create_subtitles(self, result: Dict) -> List[srt.Subtitle]:
        """Create subtitle objects with translations."""
        self.progress.update_step("Traduciendo y creando subtítulos...",
                                total=len(result["segments"]))

        subtitles = []
        for i, seg in enumerate(result["segments"], 1):
            subtitle = srt.Subtitle(
                index=i,
                start=timedelta(seconds=seg['start']),
                end=timedelta(seconds=seg['end']),
                content=self.translate_text(seg['text'])
            )
            subtitles.append(subtitle)
            self.progress.update_progress()

        return subtitles

    def add_subtitles_to_video(
        self,
        video_path: Path,
        subtitles: List[srt.Subtitle],
        output_path: Path
    ) -> Optional[Path]:
        """Add subtitles to video using ffmpeg."""
        try:
            self.progress.update_step("Agregando subtítulos al video...")

            # Guardar subtítulos en archivo temporal
            srt_path = self.config.temp_dir / "subtitles.srt"
            with open(srt_path, 'w', encoding='utf-8') as f:
                f.write(srt.compose(subtitles))

            # Usar ffmpeg para agregar subtítulos
            command = [
                'ffmpeg', '-i', str(video_path),
                '-vf', f'subtitles={srt_path}:force_style=\'FontName={self.config.subtitle_font},FontSize={self.config.subtitle_size},PrimaryColour=&HFFFFFF,OutlineColour=&H000000,BorderStyle=3\'',
                '-c:a', 'copy',
                '-y',  # Sobrescribir archivo si existe
                str(output_path)
            ]

            process = subprocess.Popen(
                command,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                universal_newlines=True
            )

            _, stderr = process.communicate()

            if process.returncode != 0:
                raise Exception(f"FFmpeg error: {stderr}")

            return output_path

        except Exception as e:
            logging.error(f"Error adding subtitles to {video_path}: {str(e)}")
            return None

    def process_video(self, url: str) -> bool:
        """Process a single video with complete workflow."""
        try:
            # Crear directorio temporal único para este video
            video_temp_dir = self.config.temp_dir / "current_video"
            video_temp_dir.mkdir(parents=True, exist_ok=True)

            # Descargar video
            video_path = self.download_video(url, video_temp_dir)
            if not video_path:
                return False

            # Extraer audio
            audio_path = video_temp_dir / "audio.wav"
            if not self.extract_audio(video_path, audio_path):
                return False

            # Transcribir audio
            result = self.transcribe_audio(audio_path)
            if not result:
                return False

            # Crear subtítulos
            subtitles = self.create_subtitles(result)

            # Crear video final
            output_filename = f"{video_path.stem}_subtitulado{video_path.suffix}"
            output_path = self.config.output_dir / output_filename
            result = self.add_subtitles_to_video(video_path, subtitles, output_path)

            # Limpiar archivos temporales
            self.cleanup_temp_files(video_temp_dir)

            if result:
                print(f"\nVideo procesado exitosamente: {output_path}")
                return True
            return False

        except Exception as e:
            logging.error(f"Error processing video {url}: {str(e)}")
            return False
        finally:
            self.progress.close()

    def cleanup_temp_files(self, directory: Path):
        """Clean up temporary files and directories."""
        try:
            if directory.exists():
                shutil.rmtree(directory)
        except Exception as e:
            logging.warning(f"Error cleaning up {directory}: {str(e)}")

def main():
    # Configurar el procesador
    config = VideoProcessingConfig(
        output_dir=Path("videos_procesados"),
        temp_dir=Path("temp"),
        video_quality="720p"
    )

    # Crear instancia del procesador
    processor = VideoProcessor(config)

    # URL del video a procesar
    video_url = "https://www.youtube.com/watch?v=7WlwWhegq-s"  # Reemplazar con tu URL

    # Procesar el video
    processor.process_video(video_url)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.10/dist-packages/pyannote/audio/pipelines/speaker_verification.py:43: UserWarning: torchaudio._backend.get_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  backend = torchaudio.get_audio_backend()
/usr/local/lib/python3.10/dist-packages/pyannote/audio/pipelines/speaker_verification.py:45: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import (
/usr/local/lib/python3.10/dist-packages/pyannote/audio/pipelines/speaker_


>>> Descargando video...
Descargando: Stan Van Gundy - Pick & Roll Defense
Descarga completada: temp/current_video/Stan Van Gundy - Pick & Roll Defense.mp4

>>> Extrayendo audio...

>>> Transcribiendo audio...


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.4.1+cu121. Bad things might happen unless you revert torch to 1.x.
Detected language: hr (0.65) in first 30s of audio...


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at classla/wav2vec2-xls-r-parlaspeech-hr were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at classla/wav2vec2-xls-r-parlaspeech-hr and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probab


>>> Traduciendo y creando subtítulos...


Traduciendo y creando subtítulos...: 100%|██████████| 400/400 [00:46<00:00,  8.60it/s]



>>> Agregando subtítulos al video...

Video procesado exitosamente: videos_procesados/Stan Van Gundy - Pick & Roll Defense_subtitulado.mp4
